In [1]:
import pandas as pd
import json
import numpy as np

In [3]:
train_x0 = [] # 案情描述，事实部分
train_y01 = [] # imprisonment 刑期类型及长短
train_y02 = [] # 被告人，背叛罪名，金钱惩罚，相关法律条文

for line in open('CAIL_data/data_train.json', 'r'):
    item = json.loads(line)
    train_x0.append(item['fact'])    # 案情描述，事实部分 
    imprisonment = item['meta'].pop('term_of_imprisonment') # imprisonment 刑期类型及长短
    train_y01.append(imprisonment)
    train_y02.append(item['meta']) # 被告人，背叛罪名，金钱惩罚，相关法律条文

In [4]:
train_df = pd.concat([pd.DataFrame(train_x0),pd.DataFrame(train_y01), pd.DataFrame(train_y02)], axis=1)
train_df[:5]

,0,death_penalty,imprisonment,life_imprisonment,accusation,criminals,punish_of_money,relevant_articles
0,昌宁县人民检察院指控，2014年4月19日下午16时许，被告人段某驾拖车经过鸡飞乡澡塘街子，...,False,12,False,[故意伤害],[段某],0,[234]
1,"公诉机关指控,2015年11月10日晚9时许，被告人李某的妹妹李某某与被害人华某某在桦川县悦...",False,10,False,[故意伤害],[李某],0,[234]
2,贵州省平坝县人民检察院指控：2014年4月9日下午，被告人王某丁与其堂哥王4某（另案处理）假...,False,42,False,[故意伤害],[王某丁],0,"[292, 234]"
3,经审理查明：2014年5月6日14时许，被告人叶某某驾车途径赤壁市赵李桥镇胜利街涵洞时，被在...,False,12,False,[故意伤害],[叶某某],0,[234]
4,安阳县人民检察院指控：2014年4月27日上午11时许，宋某甲在安阳县吕村镇翟奇务村被告人梁...,False,12,False,[故意伤害],[梁某甲],0,[234]


In [5]:
train_df.columns

Index([                  0,     'death_penalty',      'imprisonment',
       'life_imprisonment',        'accusation',         'criminals',
         'punish_of_money', 'relevant_articles'],
      dtype='object')

In [6]:
train_facts = train_df[0]
train_label = train_df['relevant_articles']


In [7]:
train_facts[:5]

0    昌宁县人民检察院指控，2014年4月19日下午16时许，被告人段某驾拖车经过鸡飞乡澡塘街子，...
1    公诉机关指控,2015年11月10日晚9时许，被告人李某的妹妹李某某与被害人华某某在桦川县悦...
2    贵州省平坝县人民检察院指控：2014年4月9日下午，被告人王某丁与其堂哥王4某（另案处理）假...
3    经审理查明：2014年5月6日14时许，被告人叶某某驾车途径赤壁市赵李桥镇胜利街涵洞时，被在...
4    安阳县人民检察院指控：2014年4月27日上午11时许，宋某甲在安阳县吕村镇翟奇务村被告人梁...
Name: 0, dtype: object

In [7]:
import jieba
import jieba.posseg as pseg
import jieba.analyse

In [16]:
# 斯坦福分词，选取有意义的名词，专有名词，和动词
from stanfordcorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('/home/jeshe/stanford-corenlp-full-2017-06-09/', lang='zh')

In [17]:
def seg_document(document):               
    result = [i for (i, j) in nlp.pos_tag(document) if j in ['NN', 'NR', 'VV']]
    return result

In [3]:
file = open('corpus.txt','w')

i = 0
corpus = []

for fact in train_facts:
    i = i+1
    line = ' '.join(seg_document(fact))
    corpus.append(line)
    file.write(line)
    print('process %d' %i)

file.close()
nlp.close()

In [2]:
# 已经生成了该有的

# file = open('corpus1.txt','w')
# file.write(str(corpus))
# file.close()

# corpus2 = '\n'.join(corpus)
# file = open('corpus2.txt','w')
# file.write(str(corpus2))
# file.close()

# with open('corpus2.txt','r') as f:
#     data = f.read()
# f.close()
# data.split('\n')

In [ ]:
##################################################################################################################

In [21]:
valid_x0 = [] # 案情描述，事实部分
valid_y01 = [] # imprisonment 刑期类型及长短
valid_y02 = [] # 被告人，背叛罪名，金钱惩罚，相关法律条文

for line in open('CAIL_data/data_valid.json', 'r'):
    item = json.loads(line)
    valid_x0.append(item['fact'])    # 案情描述，事实部分 
    imprisonment = item['meta'].pop('term_of_imprisonment') # imprisonment 刑期类型及长短
    valid_y01.append(imprisonment)
    valid_y02.append(item['meta']) # 被告人，背叛罪名，金钱惩罚，相关法律条文

In [22]:
valid_df = pd.concat([pd.DataFrame(valid_x0),pd.DataFrame(valid_y01), pd.DataFrame(valid_y02)], axis=1)
valid_df[:5]

,0,death_penalty,imprisonment,life_imprisonment,accusation,criminals,punish_of_money,relevant_articles
0,公诉机关起诉指控，被告人张某某秘密窃取他人财物，价值2210元，××数额较大，其行为已触犯《...,False,2,False,[盗窃],[张某某],0,[264]
1,孝昌县人民检察院指控：2014年1月4日，被告人邬某在孝昌县城区2路公交车上××被害人晏某白...,False,4,False,[盗窃],[邬某],1000,[264]
2,广东省广州市南沙区人民检察院指控被告人罗某于2015年6月2日到广州市南沙区大岗镇人民路宇航...,False,9,False,[盗窃],[罗某],2000,[264]
3,公诉机关指控，2016年3月3日18时许，被告人易某某行至达州市通川区大观园公交车站附近，扒...,False,7,False,[盗窃],[易某某],1000,[264]
4,公诉机关指控：1.2015年8月20日晚上，被告人胡某甲至杭州市淳安县千岛湖镇新安东路112...,False,7,False,[盗窃],[胡某甲],0,[264]


In [26]:
valid_facts = valid_df[0]
valid_label = valid_df['relevant_articles']


In [23]:
# 斯坦福分词，选取有意义的名词，专有名词，和动词
from stanfordcorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('/home/jeshe/stanford-corenlp-full-2017-06-09/', lang='zh')

In [24]:
def seg_document(document):               
    result = [i for (i, j) in nlp.pos_tag(document) if j in ['NN', 'NR', 'VV']]
    return result

In [4]:
file = open('corpus_valid.txt','w')

i = 0
corpus_valid = []

for fact in valid_facts:
    i = i+1
    line = ' '.join(seg_document(fact))
    corpus_valid.append(line)
    file.write(line)
    print('process %d' %i)

file.close()
nlp.close()

In [31]:
# 已经生成了该有的valid data

file = open('corpus1_valid.txt','w')
file.write(str(corpus_valid))
file.close()

corpus2_valid = '\n'.join(corpus_valid)
file = open('corpus2_valid.txt','w')
file.write(str(corpus2_valid))
file.close()

with open('corpus2_valid.txt','r') as f:
    data_valid = f.read()
f.close()



In [32]:
len(data_valid.split('\n'))

17131

In [ ]:
#################################################################################################################

In [34]:
test_x0 = [] # 案情描述，事实部分
test_y01 = [] # imprisonment 刑期类型及长短
test_y02 = [] # 被告人，背叛罪名，金钱惩罚，相关法律条文

for line in open('CAIL_data/data_test.json', 'r'):
    item = json.loads(line)
    test_x0.append(item['fact'])    # 案情描述，事实部分 
    imprisonment = item['meta'].pop('term_of_imprisonment') # imprisonment 刑期类型及长短
    test_y01.append(imprisonment)
    test_y02.append(item['meta']) # 被告人，背叛罪名，金钱惩罚，相关法律条文

In [35]:
test_df = pd.concat([pd.DataFrame(test_x0),pd.DataFrame(test_y01), pd.DataFrame(test_y02)], axis=1)
test_df[:5]

,0,death_penalty,imprisonment,life_imprisonment,accusation,criminals,punish_of_money,relevant_articles
0,公诉机关指控：2016年3月28日20时许，被告人颜某在本市洪山区马湖新村足球场马路边捡拾到...,False,4,False,[盗窃],[颜某],1000,[264]
1,天津市静海县人民检察院指控，2014年5月13日上午8时许，被告人李xx在天津市静海县大邱庄...,False,0,False,[盗窃],[李xx],0,[264]
2,永顺县人民检察院指控，2014年1月11日，被告人李某某与彭某某（另案处理）在永顺县塔卧镇“...,False,144,False,[强奸],[李某某],0,[236]
3,公诉机关起诉书指控：2016年11月17日凌晨1时许，被告人周某在本县武康街道营盘小区131...,False,6,False,[盗窃],[周某],5000,[264]
4,大名县人民检察院起诉书指控，2014年3月25日9时许，被告人张某在自家庄某处因故与本村席某...,False,21,False,[故意伤害],[张某],0,[234]


In [36]:
len(test_df)

32508

In [37]:
test_facts = test_df[0]
test_label = test_df['relevant_articles']

In [42]:
# 斯坦福分词，选取有意义的名词，专有名词，和动词
from stanfordcorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('/home/jeshe/stanford-corenlp-full-2017-06-09/', lang='zh')

In [43]:
def seg_document(document):               
    result = [i for (i, j) in nlp.pos_tag(document) if j in ['NN', 'NR', 'VV']]
    return result

In [5]:
file = open('corpus_test.txt','w')

i = 0
corpus_test = []

for fact in test_facts:
    i = i+1
    line = ' '.join(seg_document(fact))
    corpus_test.append(line)
    file.write(line)
    print('process %d' %i)

file.close()
nlp.close()

In [45]:
# 已经生成了该有的valid data

file = open('corpus1_test.txt','w')
file.write(str(corpus_test))
file.close()

corpus2_test = '\n'.join(corpus_test)
file = open('corpus2_test.txt','w')
file.write(str(corpus2_test))
file.close()

with open('corpus2_test.txt','r') as f:
    data_test = f.read()
f.close()

In [46]:
len(data_test.split('\n'))

32508

In [10]:
#!/usr/bin/python
import sys
import numpy as np
import gensim

from gensim import models

from gensim.models.doc2vec import Doc2Vec, TaggedDocument#,LabeledSentence
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split 


#LabeledSentence = gensim.models.doc2vec.LabeledSentence

In [84]:
train_facts = train_df[0]
train_label = train_df['relevant_articles']

In [83]:
# train_label = np.array(train_label)
# train_label = list(train_label)
# # 检查各个案例中relevant_articles会有几个，最高可能一起案件触犯了12个条文
# relevant_articles_no = [len(i) for i in train_df['relevant_articles']]
# np.unique(relevant_articles_no)

# # 统计最想相关的几个法律条文， 分别是347， 264， 383， 234， 133
# rel_art_stas = []
# for j in train_df['relevant_articles']: #j遍历154592个item in train_df['relevant_articles']
#     for i in j: #每个案件中会有几个罪行
#         rel_art_stas.append(i)
        
# rel_art_stas = pd.Series(rel_art_stas)
# #rel_art_stas.value_counts()[:10]

In [88]:
train_facts = list(train_facts)
train_label = list(train_label)

In [87]:
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer

train_arr_label= MultiLabelBinarizer().fit_transform(train_label)

In [142]:
x_training = train_facts
y_label = train_arr_label

In [143]:
x_training = corpus

In [112]:
punctuation = """.,?!:;(){}[]"""

#treat punctuation as individual words
for c in punctuation:
    corpus = [z.replace(c, ' %s '%c) for z in corpus]
x_training = [z.split() for z in corpus]


In [137]:
len(x_training), y_label.shape

(154592, (154592, 183))

In [132]:
def random_int_list(start, stop, length):
    start, stop = (int(start), int(stop)) if start <= stop else (int(stop), int(start))
    length = int(abs(length)) if length else 0
    random_list = []
    for i in range(length):
        random_list.append(random.randint(start, stop))
    return random_list

In [133]:
import random

sli = random_int_list(1, 154592, 5000) 

In [147]:
x_training_sub = [x_training[i] for i in sli]
y_label_sub = [list(y_label[i]) for i in sli]

In [153]:
sum(y_label_sub[1])

1

In [149]:
len(x_training_sub), len(y_label_sub)

(5000, 5000)

In [156]:
x_train, x_test, y_train, y_test = train_test_split(x_training_sub, y_label_sub, test_size=0.2)

In [158]:
len(x_train), len(x_test), len(y_train), len(y_test)#, #sum(y_train), len(y_test), sum(y_test)

(4000, 1000, 4000, 1000)

In [159]:
def TaggedReviews(reviews, label_type):
        labelized = []
        for i,v in enumerate(reviews):
            label = '%s_%s'%(label_type,i)
            labelized.append(TaggedDocument(v, [label]))
        return labelized

In [160]:
x_train2 = TaggedReviews(x_train, 'TRAIN')
x_test2 = TaggedReviews(x_test, 'TEST')

In [161]:
# Distributed Memory (DM) 和 Distributed bag of words (DBOW) 模型的建立

size = 400


model_dm = gensim.models.Doc2Vec(min_count=1, window=10, vector_size=size, sample=1e-3, 
                                 negative=5, workers=3)

model_dbow = gensim.models.Doc2Vec(min_count=1, window=10, vector_size=size, sample=1e-3, 
                                   negative=5, dm=0, workers=3)

In [162]:
#build model dictionary using all sentence data

model_dm.build_vocab(x_train2+x_test2)
model_dbow.build_vocab(x_train2+ x_test2)

In [163]:
#进行多次重复训练，每一次都需要对训练数据重新打乱，以提高精度

epoch_num = 1

for epoch in range(epoch_num):
    print(epoch)
    perm = list(np.random.permutation(len(x_train2)))
    x_train3 = [x_train2[i] for i in perm]
    model_dm.train(x_train3,total_examples = model_dm.corpus_count, 
                   epochs=model_dm.epochs)
    model_dbow.train(x_train3,total_examples = model_dbow.corpus_count, 
                     epochs=model_dbow.epochs)

0


In [164]:
##读取向量
def getVecs(model, corpus, size):
    vecs = [np.array(model.docvecs[z.tags[0]]).reshape((1, size)) for z in corpus]
    return np.concatenate(vecs)

In [165]:
##将训练完成的数据转换为vectors
def get_vectors(model_dm,model_dbow,x_train, x_test, size = 400):

    #获取训练数据集的文档向量
    train_vecs_dm = getVecs(model_dm, x_train, size)
    train_vecs_dbow = getVecs(model_dbow, x_train, size)
    train_vecs = np.hstack((train_vecs_dm, train_vecs_dbow))
    #获取测试数据集的文档向量
    test_vecs_dm = getVecs(model_dm, x_test, size)
    test_vecs_dbow = getVecs(model_dbow, x_test, size)
    test_vecs = np.hstack((test_vecs_dm, test_vecs_dbow))

    return train_vecs,test_vecs

In [166]:
train_vecs,test_vecs = get_vectors(model_dm,model_dbow, x_train2, x_test2)

In [167]:
train_vecs.shape, test_vecs.shape

((4000, 800), (1000, 800))

In [168]:
np.savetxt('train_vecs.csv', train_vecs, delimiter= ',')

In [169]:
np.savetxt('y_train.csv', y_train, delimiter= ',')

In [170]:
np.savetxt('test_vecs.csv', test_vecs, delimiter= ',')

In [171]:
np.savetxt('y_test.csv', y_test, delimiter= ',')

In [172]:
import numpy as np
train_vecs = np.loadtxt('train_vecs.csv', dtype = np.float, delimiter = ',')
y_train = np.loadtxt('y_train.csv', dtype = np.float, delimiter = ',')
test_vecs = np.loadtxt('test_vecs.csv', dtype = np.float, delimiter = ',')
y_test = np.loadtxt('y_test.csv', dtype = np.float, delimiter = ',')

In [173]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

Using TensorFlow backend.


In [189]:
# set parameters:
max_features = 1000
maxlen = 800
batch_size = 32
embedding_dims = 500
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

In [175]:
y_test.shape,y_train.shape, train_vecs.shape,test_vecs.shape

((1000, 183), (4000, 183), (4000, 800), (1000, 800))

In [193]:
from sklearn import preprocessing
import numpy as np
min_max_scaler = preprocessing.MinMaxScaler()
train_vecs = min_max_scaler.fit_transform(train_vecs)*999
train_vecs = train_vecs.astype(np.int)
test_vecs = min_max_scaler.fit_transform(test_vecs)*999
test_vecs = test_vecs.astype(np.int)

/home/jeshe/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [194]:
import keras.backend as K
def recall(y_true, y_pred):
    
    c1 = K.sum(K.round(K.clip(y_true*y_pred,0,1)))
    c3 = K.sum(K.round(K.clip(y_true, 0,1)))
    return c1/c3

In [199]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [200]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(input_dim = max_features,
                    output_dim =embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(183))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy',f1])



# model.compile(loss='binary_crossentropy',
#           optimizer= "adam",
#           metrics=[f1])

Build model...


In [201]:
model.fit(train_vecs, y_train,
          batch_size=batch_size,
          epochs=2, validation_data=(test_vecs, y_test))

Train on 4000 samples, validate on 1000 samples
Epoch 1/2
4000/4000 [==============================] - 808s 202ms/step - loss: 0.0988 - acc: 0.9760 - f1: 0.0019 - val_loss: 0.0437 - val_acc: 0.9929 - val_f1: 0.0000e+00
Epoch 2/2
4000/4000 [==============================] - 523s 131ms/step - loss: 0.0394 - acc: 0.9928 - f1: 0.0000e+00 - val_loss: 0.0433 - val_acc: 0.9929 - val_f1: 0.0000e+00


In [197]:
a = model.predict(test_vecs)